In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gdown
import gdown

file_id = '1M18xmxEKZ-ALyE6DD3e7ih93ZgF99qTG'
output = 'data_final.zip'
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1M18xmxEKZ-ALyE6DD3e7ih93ZgF99qTG
From (redirected): https://drive.google.com/uc?id=1M18xmxEKZ-ALyE6DD3e7ih93ZgF99qTG&confirm=t&uuid=cd37e08b-db92-4995-9cc3-88ba75c15b93
To: /content/data_final.zip
100%|██████████| 191M/191M [00:01<00:00, 177MB/s]


'data_final.zip'

In [ ]:
import zipfile

with zipfile.ZipFile('data_final.zip', 'r') as zip_ref:
    zip_ref.extractall('data_final')
print("✅ Data berhasil diekstrak ke folder 'data_final/'")

✅ Data berhasil diekstrak ke folder 'data_final/'


In [ ]:
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [ ]:
# Paths dataset
train_dir = '/content/data_final/content/data_final_split/train'
val_dir = '/content/data_final/content/data_final_split/val'
test_dir = '/content/data_final/content/data_final_split/test'

In [ ]:
# --- Set random seed agar hasil reproducible ---
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed(42)

In [ ]:
# --- Data generators ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.5, 1.5],
    channel_shift_range=20.0,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Cek jumlah kelas dari generator latihan
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

num_classes = train_gen.num_classes # Mendapatkan jumlah kelas secara dinamis

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

Found 14700 images belonging to 12 classes.
Found 3144 images belonging to 12 classes.
Found 3156 images belonging to 12 classes.


In [ ]:
print(f"Jumlah kelas yang terdeteksi oleh generator: {num_classes}")
print(f"Mapping kelas ke indeks: {train_gen.class_indices}")

Jumlah kelas yang terdeteksi oleh generator: 12
Mapping kelas ke indeks: {'Baterai': 0, 'Daun': 1, 'Elektronik': 2, 'Kaca': 3, 'Kardus': 4, 'Kertas': 5, 'Lampu': 6, 'Logam': 7, 'Pakaian': 8, 'Plastik': 9, 'Sampah Makanan': 10, 'Sterofom': 11}


# MODEL 1

In [ ]:
# --- Arsitektur Model ---
model = Sequential([
    # Blok 1
    Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(224,224,3), kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.3),

    # Blok 2 - Lebih Banyak Filter
    Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.4),

    # Blok 3 - Lebih Banyak Filter
    Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.5),

    # Blok 4 (Baru - dengan filter lebih besar)
    Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Conv2D(512, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.5),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.0001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax') # Menggunakan num_classes yang didapat dari generator
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# --- Callback untuk training ---
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 28, 28, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 56,081,996 (213.94 MB)

 Trainable params: 56,077,132 (213.92 MB)

 Non-trainable params: 4,864 (19.00 KB)

In [ ]:
# Training model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 330s 725ms/step - accuracy: 0.1686 - loss: 3.4797 - val_accuracy: 0.1948 - val_loss: 3.1125 - learning_rate: 0.0010
Epoch 2/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 243s 617ms/step - accuracy: 0.2752 - loss: 2.9661 - val_accuracy: 0.3648 - val_loss: 2.5806 - learning_rate: 0.0010
Epoch 3/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 244s 618ms/step - accuracy: 0.3130 - loss: 2.7627 - val_accuracy: 0.3885 - val_loss: 2.4295 - learning_rate: 0.0010
Epoch 4/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 242s 613ms/step - accuracy: 0.3342 - loss: 2.6227 - val_accuracy: 0.2348 - val_loss: 3.4315 - learning_rate: 0.0010
Epoch 5/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 234s 592ms/step - accuracy: 0.3599 - loss: 2.5165 - val_accuracy: 0.3911 - val_loss: 2.4174 - learning_rate: 0.0010
Epoch 6/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 239s 606ms/step - accuracy: 0.3644 - loss: 2.4493 - val_accuracy: 0.2919 - val_loss: 2.7835 - learning_rate: 0.0010
Epoch 7/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 258s 596ms/step - accura

# MODEL 2

In [ ]:
# --- Data Augmentation Layer ---
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Fungsi Membangun Model CNN dari Awal ---
def build_model6_from_scratch(input_shape, num_classes, data_augmentation_layer):
    inputs = layers.Input(shape=input_shape, name="input_layer")
    x = data_augmentation_layer(inputs)

    # Block 1
    x = layers.Conv2D(32, (3, 3), padding='same', name="conv1a")(x)
    x = layers.BatchNormalization(name="bn1a")(x)
    x = layers.Activation('relu', name="relu1a")(x)
    x = layers.Conv2D(32, (3, 3), padding='same', name="conv1b")(x)
    x = layers.BatchNormalization(name="bn1b")(x)
    x = layers.Activation('relu', name="relu1b")(x)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    x = layers.Dropout(0.25, name="drop1")(x)

    # Block 2
    x = layers.Conv2D(64, (3, 3), padding='same', name="conv2a")(x)
    x = layers.BatchNormalization(name="bn2a")(x)
    x = layers.Activation('relu', name="relu2a")(x)
    x = layers.Conv2D(64, (3, 3), padding='same', name="conv2b")(x)
    x = layers.BatchNormalization(name="bn2b")(x)
    x = layers.Activation('relu', name="relu2b")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)
    x = layers.Dropout(0.25, name="drop2")(x)

    # Block 3
    x = layers.Conv2D(128, (3, 3), padding='same', name="conv3a")(x)
    x = layers.BatchNormalization(name="bn3a")(x)
    x = layers.Activation('relu', name="relu3a")(x)
    x = layers.Conv2D(128, (3, 3), padding='same', name="conv3b")(x)
    x = layers.BatchNormalization(name="bn3b")(x)
    x = layers.Activation('relu', name="relu3b")(x)
    x = layers.MaxPooling2D((2, 2), name="pool3")(x)
    x = layers.Dropout(0.3, name="drop3")(x)

    # Fully Connected Head
    x = layers.Flatten(name="flatten")(x)
    x = layers.Dense(512, name="dense1")(x)
    x = layers.BatchNormalization(name="bn_dense1")(x)
    x = layers.Activation('relu', name="relu_dense1")(x)
    x = layers.Dropout(0.5, name="drop_dense1")(x)

    outputs = layers.Dense(num_classes, activation='softmax', name="output_layer")(x)

    model = models.Model(inputs, outputs, name="model6_from_scratch")
    return model

input_shape = (224, 224, 3)
model6 = build_model6_from_scratch(input_shape, num_classes, data_augmentation)

# --- Kompilasi Model ---
initial_learning_rate = 1e-3
optimizer = optimizers.Adam(learning_rate=initial_learning_rate)

model6.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model6.summary()

# --- Callbacks ---
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=7,
    min_lr=1e-6,
    verbose=1
)

callbacks_list = [early_stopping, reduce_lr]

# --- Training Model ---
epochs = 100

print("Memulai training model6 (from scratch)...")
history = model6.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen,
    callbacks=callbacks_list,
    verbose=1
)

Model: "model6_from_scratch"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1a (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1a (BatchNormalization)       │ (None, 224, 224, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu1a (Activation)             │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1b (Conv2D)                 │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1b (BatchNormalization)       │ (None, 224, 224, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu1b (Activation)             │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2a (Conv2D)                 │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2a (BatchNormalization)       │ (None, 112, 112, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu2a (Activation)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2b (Conv2D)                 │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2b (BatchNormalization)       │ (None, 112, 112, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu2b (Activation)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3a (Conv2D)                 │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3a (BatchNormalization)       │ (None, 56, 56, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu3a (Activation)             │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3b (Conv2D)                 │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3b (BatchNormalization)       │ (None, 56, 56, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu3b (Activation)             │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 28, 28, 128)    │             

 Total params: 51,677,740 (197.13 MB)

 Trainable params: 51,675,820 (197.13 MB)

 Non-trainable params: 1,920 (7.50 KB)

Memulai training model6 (from scratch)...
Epoch 1/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 227s 561ms/step - accuracy: 0.2009 - loss: 2.5809 - val_accuracy: 0.2107 - val_loss: 2.2116 - learning_rate: 0.0010
Epoch 2/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 225s 570ms/step - accuracy: 0.2850 - loss: 2.1430 - val_accuracy: 0.3663 - val_loss: 1.9286 - learning_rate: 0.0010
Epoch 3/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 224s 569ms/step - accuracy: 0.3266 - loss: 1.9983 - val_accuracy: 0.2267 - val_loss: 3.0336 - learning_rate: 0.0010
Epoch 4/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 220s 557ms/step - accuracy: 0.3430 - loss: 1.9194 - val_accuracy: 0.2804 - val_loss: 2.5738 - learning_rate: 0.0010
Epoch 5/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 224s 567ms/step - accuracy: 0.3745 - loss: 1.8814 - val_accuracy: 0.2878 - val_loss: 2.7649 - learning_rate: 0.0010
Epoch 6/100
394/394 ━━━━━━━━━━━━━━━━━━━━ 226s 572ms/step - accuracy: 0.3890 - loss: 1.8169 - val_accuracy: 0.4163 - val_loss: 1.7424 - learning_rate: 0.0010
Epoch 7/100
394/

In [ ]:
# Evaluasi data test
test_loss, test_acc = model6.evaluate(test_gen, steps=test_gen.samples // test_gen.batch_size)
print(f"Akurasi test: {test_acc * 100:.2f}%")

84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.5933 - loss: 1.2022
Akurasi test: 60.04%


# MODEL 3

In [ ]:
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # Blok 1
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    BatchNormalization(),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    # Blok 2
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),

    # Blok 3
    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    # Fully connected
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)

# Train model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[reduce_lr, earlystop]
)

Epoch 1/25
394/394 ━━━━━━━━━━━━━━━━━━━━ 224s 516ms/step - accuracy: 0.1932 - loss: 2.7774 - val_accuracy: 0.1044 - val_loss: 5.8542 - learning_rate: 1.0000e-04
Epoch 2/25
394/394 ━━━━━━━━━━━━━━━━━━━━ 187s 476ms/step - accuracy: 0.3047 - loss: 2.2052 - val_accuracy: 0.3174 - val_loss: 3.4066 - learning_rate: 1.0000e-04
Epoch 3/25
394/394 ━━━━━━━━━━━━━━━━━━━━ 187s 474ms/step - accuracy: 0.3451 - loss: 2.0481 - val_accuracy: 0.3159 - val_loss: 4.8163 - learning_rate: 1.0000e-04
Epoch 4/25
394/394 ━━━━━━━━━━━━━━━━━━━━ 202s 474ms/step - accuracy: 0.3494 - loss: 2.0018 - val_accuracy: 0.3941 - val_loss: 4.9658 - learning_rate: 1.0000e-04
Epoch 5/25
394/394 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step - accuracy: 0.3895 - loss: 1.9153
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
394/394 ━━━━━━━━━━━━━━━━━━━━ 201s 472ms/step - accuracy: 0.3895 - loss: 1.9153 - val_accuracy: 0.3211 - val_loss: 4.1749 - learning_rate: 1.0000e-04
Epoch 6/25
394/394 ━━━━━━━━━━━━━━━━━━━━ 186s 47

In [ ]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Akurasi pada data test: {test_acc * 100:.2f}%")

85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.2646 - loss: 4.5097
Akurasi pada data test: 32.22%


# MODEL 4

In [ ]:
# Model CNN sederhana
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callback
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

# Training model
history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 230s 485ms/step - accuracy: 0.1584 - loss: 2.3842 - val_accuracy: 0.2793 - val_loss: 2.1098 - learning_rate: 0.0010
Epoch 2/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 217s 472ms/step - accuracy: 0.2574 - loss: 2.1734 - val_accuracy: 0.3502 - val_loss: 1.9245 - learning_rate: 0.0010
Epoch 3/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 260s 468ms/step - accuracy: 0.2844 - loss: 2.0874 - val_accuracy: 0.3451 - val_loss: 2.0045 - learning_rate: 0.0010
Epoch 4/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 215s 467ms/step - accuracy: 0.3101 - loss: 2.0322 - val_accuracy: 0.3950 - val_loss: 1.8147 - learning_rate: 0.0010
Epoch 5/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 262s 467ms/step - accuracy: 0.3273 - loss: 1.9823 - val_accuracy: 0.4275 - val_loss: 1.7108 - learning_rate: 0.0010
Epoch 6/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 214s 464ms/step - accuracy: 0.3488 - loss: 1.9302 - val_accuracy: 0.4240 - val_loss: 1.6908 - learning_rate: 0.0010
Epoch 7/20
460/460 ━━━━━━━━━━━━━━━━━━━━ 212s 460ms/step - accura

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy:.2%}")

82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.6010 - loss: 1.2080
Test Accuracy: 61.92%
